<a href="https://colab.research.google.com/github/theharshithh/onfinace-assignment/blob/main/cohere_embeds4_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
!pip -q install langchain openai tiktoken chromadb langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.5/111.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install langchain_cohere

In [13]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_cohere import CohereEmbeddings


def read_doc(dir):
    file_loader = PyPDFDirectoryLoader(dir)
    docs = file_loader.load()
    return docs

loading data to /dataset dir

In [ ]:
!pip install pypdf sentence_transformers

In [ ]:
primary_data = read_doc('dataset/')
primary_data

creating chunks for the `primary_data`

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
spilit_texts = text_splitter.split_documents(primary_data)

In [ ]:
!pip install cohere

## create the embeddings using hf ede by baai

In [11]:
# #hugging face bge embeddings
# model_name = "Cohere/Cohere-embed-english-v3.0"
# encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

# model_norm = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cuda'},
#     encode_kwargs=encode_kwargs
# )

change of embeddings from `OpenAIEmbeddings()` to bge-embeddings

In [17]:
persist_directory = 'db-cohere'
from google.colab import userdata
cohere_api_key = userdata.get('COHERE_API_KEY')

#cohere embeddings -> converts texts to vecs
embedding = CohereEmbeddings(cohere_api_key=cohere_api_key,model="embed-english-v3.0")

#load the spilt chunk vectors into the vectordb
vectordb = Chroma.from_documents(documents=spilit_texts, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()
vectordb = None
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [18]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [19]:

user_query = 'Considering 2 wheeler demand, who seems to be the segment winner?'

In [20]:
retriever = vectordb.as_retriever()
docs = retriever.get_relevant_documents(user_query)
#getting the relevant docs for the user query
docs

[Document(page_content='5% YoY , largely due to  the impact of  unseasonal rains and low yield s for key \ncrops  such as oni on. In Mar’23, overall wholesale volumes for \nPV/CV/tractor/3Ws are estimated to grow by 1%/ 1.5%/1 %/8% YoY, while 2Ws \nare likely to decline 4% YoY.  \n\uf06e 2Ws:  Retails are expected to grow 4-6% YoY in Mar ’23 since urban demand is \nsteady and inquiries have improved in certain states during the ongoing Navratri \nseason . However, overall sales  are still do wn by 15 -17% v /s the pre-Covid  level \nas rural demand remain s weak.  Enquiry levels during the key festival s (Gudi \nPadwa and Ugadi) varied greatly among  states.  We have noted 2-6% YoY growth \nin enquiry levels in states like Gujarat, Delhi and Karnataka , while  the same \ndeclined by 4-8% in other key states like UP, Bihar  and Maharashtra . The i nitial \nresponse to HMCL’ s new launch Xoom (110 cc scooter) has been positi ve as it has \naccounted for 25-30% of the overall enquiries in

In [21]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [22]:
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

### LLM chain

In [23]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(openai_api_key=openai_api_key,model="gpt-4-turbo", temperature=1.2 ), chain_type="stuff", retriever=retriever, return_source_documents=True)

In [24]:
def process_llm_response(llm_response):
    print("Decision:")
    print(llm_response['result'])

    print("\n Source Documents:")
    for source in llm_response["source_documents"]:
        print("- " + source.metadata['source'])

    print("\nRelevant Data:")
    for source in llm_response["source_documents"]:
        print(source.page_content)

In [25]:
behaviour = '''Think like an expert finance consultant from a top tier firm. Do all the research and analysis for this document.'''
prompt_template = 'Keep the answer crisp. Give me as much numbers as you can. Every Statement of yours should be backed with data'
stop_prompt = f'Dont hallucinate, if u dont know the answer, tell you dont know'
query = f"Here is the user query : {user_query}. Remember this: {behaviour}. Give me the answers like this: {prompt_template}. Keep this in mind - {stop_prompt}"
llm_response = qa_chain(query)
# llm_response
process_llm_response(llm_response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Decision:
Based on the data provided for 2-wheeler segments from various manufacturers for the specified period:

1. **Hero MotoCorp** shows positive year-on-year growth with units sold increasing from 49,44,148 to 52,43,369 (a **6.1%** increase). This indicates resilient demand, positioning Hero MotoCorp strongly in terms of sales volume in the 2-wheeler segment.

2. **TVS Motor** similarly demonstrated substantial growth. They showed an especially strong performance in scooters, with numbers increasing from 9,21,939 to 12,97,275, exhibiting an impressive **40.7%** year-on-year growth. Motorcycle sales remained almost flat while mopeds decreased by **-5.0%**. However, due to the strong scooter numbers, TVS Motor allows for considerable brand perception and growth.

3. **Bajaj Auto**, on the other hand, experienced a decrease in 2-wheeler sales, dropping from 43,08,433 to 39,16,077, marking a drop of **-9.1%** year-on-year which indicates a relatively weaker market position in comparis